In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# 讀取數據
data = pd.read_csv('your_dataset.csv')  # 替換為你的數據文件路徑

# 提取特徵和目標
features = data.drop('churn', axis=1)
target = data['churn']

# 分割數據為訓練集和測試集
train_X, test_X, train_y, test_y = train_test_split(features, target, test_size=0.2, random_state=42)

# 特徵標準化
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)

# 轉換為 PyTorch 張量
train_X_tensor = torch.FloatTensor(train_X)
test_X_tensor = torch.FloatTensor(test_X)
train_y_tensor = torch.FloatTensor(train_y.values).unsqueeze(1)  # 將維度從 (N,) 轉換為 (N, 1)
test_y_tensor = torch.FloatTensor(test_y.values).unsqueeze(1)

# 創建 DataLoader
train_dataset = TensorDataset(train_X_tensor, train_y_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 定義 ANN 分類模型
class ANNClassifier(nn.Module):
    def __init__(self, input_size):
        super(ANNClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

input_size = train_X.shape[1]  # 輸入特徵數量
model = ANNClassifier(input_size)

# 定義損失函數和優化器
criterion = nn.BCELoss()  # 二元交叉熵損失
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練模型
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # 在測試集上評估準確率
    model.eval()
    with torch.no_grad():
        test_outputs = model(test_X_tensor)
        predictions = (test_outputs >= 0.5).float()
        accuracy = accuracy_score(test_y_tensor, predictions)
        print(f'Epoch [{epoch+1}/{num_epochs}] - Test Accuracy: {accuracy:.4f}')
